In [2]:
%pylab
from __future__ import print_function

%matplotlib inline

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from __future__ import print_function

import csv
from src import main, utils
import sys

projects = main.load_projects()
interests = ['argouml v0.26.2 method', 'mucommander v0.8.5 method']

p = list()
for project in projects:
    if ' '.join([project.name, project.version, project.level]) not in interests:
        continue
    p.append(project)

In [4]:
project = p[1]
print(project)
repos = main.load_repos(project)
goldsets = main.load_goldsets(project)

queries = main.create_queries(project)
snapshot = main.create_release_corpus(project, repos)
changesets = main.create_corpus(project, repos, main.ChangesetCorpus, use_level=False)

snapshot_lda, _  = main.create_lda_model(project, snapshot, None, "Release", use_level=True)
changeset_lda, _ = main.create_lda_model(project, changesets, None, "Changeset", use_level=False)

Project(name='mucommander', printable_name='muCommander', version='v0.8.5', level='method', ref='refs/tags/release_0_8_5', alpha=None, eta=None, passes=5, iterations=1000, num_topics=500, src_url='http://trac.mucommander.com/changeset/3794/tags/release_0_8_5?old_path=%2F&format=zip', data_path='data/mucommander/', full_path='data/mucommander/v0.8.5/', src_path='data/mucommander/v0.8.5/src')


In [5]:
q = dict()
for query, topics in main.get_topics(snapshot_lda, queries, by_ids=["37", "142"], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q[query[0]] = topics

qc = dict()
for query, topics in main.get_topics(changeset_lda, queries, by_ids=["37", "142"], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc[query[0]] = topics
    
g37 = ["com.mucommander.ui.main.menu.MainMenuBar.MainMenuBar(MainFrame)",
       "com.mucommander.ui.action.impl.MinimizeWindowAction.performAction()"]
g142 = ["com.mucommander.ui.main.DrivePopupButton.RefreshDriveNamesAndIcons.run()",
        "com.mucommander.ui.main.StatusBar.VolumeSpaceLabel.paint(Graphics)"]

q37 = dict()
for query, topics in main.get_topics(snapshot_lda, snapshot, by_ids=[g37[0]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q37[query[0]] = topics


qc37 = dict()
for query, topics in main.get_topics(changeset_lda, snapshot, by_ids=[g37[1]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc37[query[0]] = topics

q142 = dict()
for query, topics in main.get_topics(snapshot_lda, snapshot, by_ids=[g142[0]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q142[query[0]] = topics


qc142 = dict()
for query, topics in main.get_topics(changeset_lda, snapshot, by_ids=[g142[1]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc142[query[0]] = topics

In [6]:
def printer(q, model):
    for qid, topics in q.items():
        print(qid, "num topics:", len(topics))
        for t in topics:
            if len(topics) <= 3 or t[1] >= 0.1:
                print("    ", t[0], t[1])
                for word in model.show_topic(t[0]):
                    if word[0]>= 0.05:
                        print("        {0} {1}".format(*word))

# Ranks
- mucommander 37
    - snapshot 1 com.mucommander.ui.main.menu.MainMenuBar.MainMenuBar(MainFrame)
    - changeset 303 com.mucommander.ui.action.impl.MinimizeWindowAction.performAction()
- mucommander 142
    - changeset 1 com.mucommander.ui.main.StatusBar.VolumeSpaceLabel.paint(Graphics)
    - snapshot 536 com.mucommander.ui.main.DrivePopupButton.RefreshDriveNamesAndIcons.run()

# Query words (preprocessed)

In [11]:
### queries.metadata = True
qs = list(filter(lambda x: x[1][0] in ["37", "142"], queries))

for query, metadata in qs:
    doc = sorted(query, key=lambda x: x[1], reverse=True)
    words = [ ( freq, queries.id2word[wid] ) for wid, freq in doc ]
    print(metadata[0], "num words:", len(words))
    for word in words:
        print("        {1} ({0})".format(*word))

37 num words: 10
        mac (3)
        menu (3)
        window (2)
        minimize (2)
        missing (2)
        zoom (2)
        application (1)
        mimic (1)
        options (1)
        standard (1)
142 num words: 18
        java (7)
        drive (6)
        icons (3)
        button (3)
        popup (3)
        names (3)
        refresh (3)
        exception (2)
        hashtable (2)
        main (1)
        lang (1)
        mucommander (1)
        com (1)
        util (1)
        npe (1)
        pointer (1)
        run (1)
        thread (1)


# Query topics

In [8]:
print("Snapshot")
printer(q, snapshot_lda)

print()
print("Changeset")
printer(qc, changeset_lda)

Snapshot
142 num topics: 12
     486 0.190691474666
        0.225407162454 java
        0.0667966365939 version
     129 0.15194579898
        0.0836325698865 drive
     197 0.138616099833
        0.335195301674 component
        0.151223520535 popup
        0.0779508804518 components
        0.0601404620095 toolbar
        0.0592926573171 visible
     433 0.106878525059
        0.0705262831764 folder
        0.0665606935383 file
37 num topics: 8
     280 0.298581059716
        0.223045774075 menu
        0.0922777930903 item
        0.0766523325734 action
        0.0556715337321 mnemonic
        0.0534673653631 add
     21 0.161671782624
        0.0887366858441 mac
        0.0883327991072 href
        0.084118802622 comp
        0.0704069827643 data
        0.0661999626631 string
        0.0655555700563 component
        0.0590099396551 length
     437 0.111222222222
        0.233421011023 element
        0.133907408373 attr
        0.0699603578927 xml
        0.0632732165314 exceptio

# Issue 37 top method topics

In [9]:
print("Snapshot")
printer(q37, snapshot_lda)

print()
print("Changeset")
printer(qc37, changeset_lda)

Snapshot
com.mucommander.ui.main.menu.MainMenuBar.MainMenuBar(MainFrame) num topics: 2
     280 0.895554169421
        0.223045774075 menu
        0.0922777930903 item
        0.0766523325734 action
        0.0556715337321 mnemonic
        0.0534673653631 add
     492 0.104046311908
        0.405491679193 action
        0.0963417533379 descriptor
        0.0667780136219 register
        0.0570310172299 factory

Changeset
com.mucommander.ui.action.impl.MinimizeWindowAction.performAction() num topics: 3
     76 0.410787090128
        0.150192555606 window
        0.135369924693 focus
        0.11055842372 frame
        0.109025490271 main
        0.0561341944788 request
     472 0.241346080919
        0.137835824374 events
        0.137234417418 mode
        0.121380838756 thread
        0.0711481876135 action
        0.062647338085 code
        0.0575339382588 separate
     392 0.237422384509
        0.127821431271 popup
        0.0687645755657 drive
        0.0681926443664 menu
       

# Issue 142  top method topics

In [10]:
print()
print("Snapshot")
printer(q142, snapshot_lda)

print()
print("Changeset")
printer(qc142, changeset_lda)


Snapshot
com.mucommander.ui.main.DrivePopupButton.RefreshDriveNamesAndIcons.run() num topics: 12
     111 0.361327898871
        0.392817788721 icon
        0.143936260431 file
     175 0.11558893496
        0.647603479703 name
        0.12470674585 string
     254 0.101027410261
        0.146340723821 file
        0.0801011442289 icons
        0.0597020707911 system
        0.0544446927419 code

Changeset
com.mucommander.ui.main.StatusBar.VolumeSpaceLabel.paint(Graphics) num topics: 7
     80 0.297134192603
        0.123220414374 space
        0.115098043062 free
        0.0713823734647 theme
        0.0578063895751 editor
     477 0.267417534046
        0.0888027061568 width
        0.0799202282068 height
        0.0733120711679 color
     307 0.245727338065
        0.289156911138 space
        0.17043758721 free
        0.135048400676 total
        0.0857215874595 info
        0.079255558935 volume
     382 0.114901973289
        0.229433565748 color
        0.117778538188 backgrou